In [ ]:
%matplotlib notebook
import sympy
from sympy.utilities.autowrap import ufuncify
import numpy
from matplotlib import pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
sympy.init_printing()

In [ ]:
sympy.var
W, T0, T1, t = sympy.var("W T0 T1 t", real=True, positive=True)
M = sympy.Matrix([[-W, 1 / T1, 0], [0, -1 / T1, 1 / T0], [W, 0, -1 / T0]])
M

In [ ]:
eigensys = M.eigenvects()

In [ ]:
evals = []
evecs = []
for val, m, vecs in eigensys:
    for i in range(m):
        evals.append(val)
        evecs.append(vecs[i])

In [ ]:
def compute_solution(W_val, T0_val, T1_val):
    sevals = [
        sympy.simplify(val.subs({
            W: W_val,
            T0: T0_val,
            T1: T1_val
        })) for val in evals
    ]
    sevecs = [
        sympy.simplify(vec.subs({
            W: W_val,
            T0: T0_val,
            T1: T1_val
        })) for vec in evecs
    ]

    N0 = sympy.Matrix([1, 0, 0])
    c0, c1, c2 = sympy.var("c0 c1 c2")
    C = sympy.Matrix([c0, c1, c2])
    A = sympy.zeros(3, 4)
    for i in range(len(sevecs)):
        for j in range(len(sevecs)):
            A[i, j] = sevecs[j][i]

    A[:, 3] = N0

    sol = sympy.solve_linear_system_LU(A, [c0, c1, c2])
    coeffs = [complex(sympy.simplify(sol[c])) for c in C]
    fundamental_solutions = [
        coeff * vec * sympy.exp(val * t)
        for coeff, val, vec in zip(coeffs, sevals, sevecs)
    ]
    solution = fundamental_solutions[0]
    for fs in fundamental_solutions[1:]:
        solution += fs
    solution = sympy.lambdify((t), sympy.simplify(solution))

    def f(t_val):
        try:
            iter(t_val)
            return numpy.array([solution(tmp)[:,0] for tmp in t_val]).transpose()
        except TypeError:
            return solution(t_val)[:, 0]
    return f

In [ ]:
widget_W = widgets.FloatText(value=1., description=r"$W/\tau_1$")
widget_tau_0 = widgets.FloatText(value=1000., description=r"$\tau_0/\tau_1$")
@interact(W=widget_W, T0=widget_tau_0)
def plot(W, T0):
    N_t = compute_solution(W, T0, 1.)
    ts = numpy.linspace(0, 10, 1000)
    Ns = N_t(ts)
    Ns = numpy.real(Ns)
    fig, ax = plt.subplots(1,1)
    ax.plot(ts, Ns[0,:], label="$N_0$")
    ax.plot(ts, Ns[1,:], label="$N_1$")
    ax.plot(ts, Ns[2,:], label="$N_2$")
    ax.set_xlabel("$t$")
    ax.set_ylabel(r"$\frac{N(t)}{N(0)}$")
    ax.legend()
    ax.grid(True)
    
    fig, ax = plt.subplots(1,1)
    ax.plot(ts, Ns[2,:]-Ns[1,:])
    ax.set_xlabel("$t$")
    ax.set_ylabel(r"$\frac{1}{N(0)}\left(N_2(t)-N_1(t)\right)$")
    ax.grid()
    fig.tight_layout()